# Assignment 4 - Rakshit Sareen, Shubhanshu Surana - rs5606, ss11012

In [137]:
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt
from datetime import timedelta as td

In [138]:
data = pd.read_csv('conv.csv', infer_datetime_format = True, header = 0, parse_dates = [0])

In [139]:
data.head()

,DateTime,Bid,Ask
0,2018-01-07 22:02:03.832,1.35560,1.35616
1,2018-01-07 22:02:03.845,1.35560,1.35617
2,2018-01-07 22:02:03.851,1.35554,1.35617
3,2018-01-07 22:02:03.859,1.35554,1.35618
4,2018-01-07 22:02:04.114,1.35554,1.35619


In [140]:
orders = pd.read_csv('MMOrderSet.csv', infer_datetime_format = True, header = 0, parse_dates = [1])

In [141]:
orders.head()

,Side,Time
0,B,2018-01-07 22:04:12
1,S,2018-01-07 22:04:13
2,B,2018-01-07 22:04:14
3,S,2018-01-07 22:04:15
4,B,2018-01-07 22:04:16


In [142]:
orders.tail()

,Side,Time
85,S,2018-01-12 08:45:03
86,S,2018-01-12 08:45:04
87,S,2018-01-12 08:45:05
88,S,2018-01-12 08:45:06
89,S,2018-01-12 08:45:07


In [143]:
data['Spread'] = data['Ask'] - data['Bid']
data.head()

,DateTime,Bid,Ask,Spread
0,2018-01-07 22:02:03.832,1.35560,1.35616,0.00056
1,2018-01-07 22:02:03.845,1.35560,1.35617,0.00057
2,2018-01-07 22:02:03.851,1.35554,1.35617,0.00063
3,2018-01-07 22:02:03.859,1.35554,1.35618,0.00064
4,2018-01-07 22:02:04.114,1.35554,1.35619,0.00065


In [144]:
data['Mid'] = (data['Ask'] + data['Bid'])/2
data.head()

,DateTime,Bid,Ask,Spread,Mid
0,2018-01-07 22:02:03.832,1.35560,1.35616,0.00056,1.355880
1,2018-01-07 22:02:03.845,1.35560,1.35617,0.00057,1.355885
2,2018-01-07 22:02:03.851,1.35554,1.35617,0.00063,1.355855
3,2018-01-07 22:02:03.859,1.35554,1.35618,0.00064,1.355860
4,2018-01-07 22:02:04.114,1.35554,1.35619,0.00065,1.355865


In [146]:
def calculate_market_price(tickdata, localorders):
    WAIT_TIME = 10
    #current_time = tickdata.loc[0, 'DateTime']
    current_time = tickdata.loc[0, 'DateTime'] + td(0, 0.01)
    print(current_time)
    print(tickdata.loc[0, 'DateTime'])
    
    # for intial tick data we are in market. We can move this into the for loop 
    # just add another varible to define strategy
    current_bid_price = tickdata.loc[0, 'Bid']
    current_ask_price = tickdata.loc[0, 'Ask']
    oindex = 0
    PIP = 0.0001
    PIP_MULTI = 5
    TOT_MULTI = PIP * PIP_MULTI
    total_pnl = 0
    inventory = 0 # open position
    THRESHOLD_INVENTORY = 8 # millions
    last_order_type = None
    last_order_execution_price = None
    strategy = 'IN'
    print(len(orders))
    for index, tick in tickdata.iterrows():
        
        if current_time >= tickdata.loc[index, 'DateTime']:
            continue
        
        diff = (tick['DateTime'] - current_time).total_seconds() * 1000 # in milliseconds
        if(diff > WAIT_TIME):
            current_time = tick['DateTime']
            # we have to write a switch case here to decide our ask and bid price based on the strategy
            if(strategy == 'IN'):
                current_bid_price = tickdata.loc[index, 'Bid'] + TOT_MULTI
                current_ask_price = tickdata.loc[index, 'Ask'] - TOT_MULTI
            elif(strategy == 'AT'):
                current_bid_price = tickdata.loc[index, 'Bid']
                current_ask_price = tickdata.loc[index, 'Ask']
            else:
                current_bid_price = tickdata.loc[index, 'Bid'] - TOT_MULTI
                current_ask_price = tickdata.loc[index, 'Ask'] + TOT_MULTI
                
            data.loc[index, 'OFFER_BID'] = current_bid_price
            data.loc[index, 'OFFER_ASK'] = current_ask_price
                
        else:
            data.loc[index, 'OFFER_BID'] = current_bid_price
            data.loc[index, 'OFFER_ASK'] = current_ask_price
            
        # break out of function as all orders are executed
        if oindex == len(orders):
            print(total_pnl)
            return 
        
        order_type , order_time = localorders.loc[oindex, 'Side'], localorders.loc[oindex, 'Time']
        
        
        if(order_time >= tick['DateTime']):
            
            # here we have to also check what was the last order which was executed
            # order execution probablity is dependent on that
            # Write code here
            
            if(order_type == 'B'):
                # execute order wrt bid price
                # update pnl market-mid - market-bid-offer
                # also check if order was executed
                # open position will +1 million
                pnl = tickdata.loc[index, 'Mid'] - current_bid_price
                orders.loc[oindex, 'pnl'] = pnl
                orders.loc[oindex, 'strategy'] = strategy
                total_pnl += pnl
                oindex += 1
                inventory += 1
                last_order_type = 'B'
                last_order_execution_price = current_bid_price
            else:
                # execute order wrt to ask price -- this is a sell order
                # update pnl market-ask-offer - market-mid
                # also validate order was executed 
                # open position will be -1 million
                pnl = current_ask_price - tickdata.loc[index, 'Mid']
                orders.loc[oindex, 'pnl'] = pnl
                orders.loc[oindex, 'strategy'] = strategy
                total_pnl += pnl
                oindex += 1
                inventory -= 1
                last_order_type = 'S'
                last_order_execution_price = current_ask_price
        else:
            # i think we dont need this else
            pass# remove this line
        
        # here we have to define our strategy for next market bid and ask price based on the open position
        # check for open position if >8 million or if it is <-8 million
        # based on the values of open position we have to decide if we will be in market, at market or outside market
        
        if(inventory > THRESHOLD_INVENTORY):
            strategy = 'OUT'
        elif(inventory < -THRESHOLD_INVENTORY):
            strategy = 'OUT'
        elif(total_pnl < 0):
            strategy = 'IN'
        elif(total_pnl > 0):
            strategy = 'AT'

In [147]:
calculate_market_price(data, orders)

2018-01-07 22:02:03.842000
2018-01-07 22:02:03.832000
90
0.03954999999999598


In [148]:
data.head(12)

,DateTime,Bid,Ask,Spread,Mid,market_bid,market_ask,OFFER_BID,OFFER_ASK
0,2018-01-07 22:02:03.832,1.35560,1.35616,0.00056,1.355880,0.0,0.0,NaN,NaN
1,2018-01-07 22:02:03.845,1.35560,1.35617,0.00057,1.355885,0.0,0.0,1.35560,1.35616
2,2018-01-07 22:02:03.851,1.35554,1.35617,0.00063,1.355855,0.0,0.0,1.35560,1.35616
3,2018-01-07 22:02:03.859,1.35554,1.35618,0.00064,1.355860,0.0,0.0,1.35554,1.35618
4,2018-01-07 22:02:04.114,1.35554,1.35619,0.00065,1.355865,0.0,0.0,1.35554,1.35619
5,2018-01-07 22:02:06.440,1.35546,1.35619,0.00073,1.355825,0.0,0.0,1.35546,1.35619
6,2018-01-07 22:02:08.840,1.35546,1.35618,0.00072,1.355820,0.0,0.0,1.35546,1.35618
7,2018-01-07 22:02:09.414,1.35545,1.35618,0.00073,1.355815,0.0,0.0,1.35545,1.35618
8,2018-01-07 22:02:14.414,1.35546,1.35618,0.00072,1.355820,0.0,0.0,1.35546,1.35618
9,2018-01-07 22:02:15.432,1.35546,1.35619,0.00073,1.355825,0.0,0.0,1.35546,1.35619


In [149]:
orders.head()

,Side,Time,pnl,strategy
0,B,2018-01-07 22:04:12,0.000285,IN
1,S,2018-01-07 22:04:13,0.000305,AT
2,B,2018-01-07 22:04:14,0.000320,AT
3,S,2018-01-07 22:04:15,0.000325,AT
4,B,2018-01-07 22:04:16,0.000365,AT


In [150]:
orders.tail()

,Side,Time,pnl,strategy
85,S,2018-01-12 08:45:03,0.000700,OUT
86,S,2018-01-12 08:45:04,0.000710,OUT
87,S,2018-01-12 08:45:05,0.000645,OUT
88,S,2018-01-12 08:45:06,0.000205,AT
89,S,2018-01-12 08:45:07,0.000210,AT
